In [23]:
import pandas as pd
import numpy as np
import pickle
import warnings
from sklearn.preprocessing import LabelEncoder

%run models.ipynb
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

warnings.filterwarnings("ignore")

In [85]:
dataset = "re3d"
columns = ['sentence_length', 'root_word', 'root_index',
       'entity_1_index', 'entity_2_index', 'entity_distance',
       'no_words_before_entity_1', 'no_words_after_entity_2',
       'entity_1_root_distance', 'entity_2_root_distance', 'entity 1 name',
       'entity 2 name', 'entity_pos_1',
       'entity_pos_2', 'entity_dep_1', 'entity_dep_2', 'shortest_distance', 'et-label']

features = pd.read_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Features/Latest/features_"+dataset+".csv", index_col=0)
features = features[columns]
features = features.rename(columns = {"et-label": "label"})
features.head(10)
# features['entity_type_1'] = features['entity 1 type'].fillna("OTHER")
# features['entity_type_2'] = features['entity 2 type'].fillna("OTHER")
# features = features.drop(["entity 1 type", "entity 2 type"], axis=1)

,sentence_length,root_word,root_index,entity_1_index,entity_2_index,entity_distance,no_words_before_entity_1,no_words_after_entity_2,entity_1_root_distance,entity_2_root_distance,entity 1 name,entity 2 name,entity_pos_1,entity_pos_2,entity_dep_1,entity_dep_2,shortest_distance,label
21,23,shows,9,7,9,2,6,13,2,0,Daesh,Marea,PROPN,PROPN,pobj,pobj,2,ORG-LOC
27,31,inflicting,10,20,25,5,19,5,-10,15,other towns and areas,Syria,ADJ-NOUN-CCONJ-NOUN,PROPN,amod-conj-cc-conj,pobj,6,LOC-LOC
2,37,met,12,1,1,0,0,35,11,-11,U.S. Army General Joseph Votel,U.S. Air Force civilian employees,PROPN-PROPN-PROPN-PROPN-PROPN,PROPN-PROPN-PROPN-ADJ-NOUN,compound-compound-compound-compound-nsubj,compound-compound-nmod-amod-pobj,0,PER-ORG
0,24,shown,7,3,7,4,2,16,4,0,U.S. service members,Syria,PROPN-NOUN-NOUN,PROPN,compound-compound-pobj,pobj,4,ORG-LOC
5,40,advising,6,4,10,6,3,29,2,4,U.S. forces,Syrian Democratic Forces,PROPN-NOUN,PROPN-PROPN-PROPN,compound-nsubj,amod-compound-dobj,5,ORG-ORG
23,45,said,9,27,42,15,26,2,-18,33,Daesh,Iraq,PROPN,PROPN,dobj,pobj,10,ORG-LOC
24,45,said,9,27,44,17,26,0,-18,35,Daesh,Syria,PROPN,PROPN,dobj,conj,11,ORG-LOC
35,49,made,26,5,5,0,4,43,21,-21,the enemy,the SDF,DET-NOUN,DET-PROPN,det-pobj,det-appos,0,ORG-ORG
43,23,are,6,2,16,14,1,6,4,10,the Turks,our Coalition partners,DET-PROPN,DET-PROPN-NOUN,det-appos,poss-compound-pobj,10,ORG-ORG
32,33,echo,1,17,19,2,16,13,-16,18,citizens,Tikrit,NOUN,PROPN,pobj,pobj,4,ORG-LOC


## Feature Encoding

In [22]:
## For Entity Type Labels
ENT_TYPES = ["PER", "GPE", "LOC", "ORG"]
def recode_entity_types(value):
    if value.lower() in ["person", "per"]:
        return "PER"
    elif value.lower() in ["organisation", "org"]:
        return "ORG"
    elif value.lower() in ["gpe"]:
        return "GPE"
    elif value.lower() in ["location", "loc"]:
        return "LOC"
    else:
        return "OTHER"

In [4]:
features["entity_type_1"] = features["entity_type_1"].apply(lambda x : recode_entity_types(x.split("-")[0]))
features["entity_type_2"] = features["entity_type_2"].apply(lambda x : recode_entity_types(x.split("-")[0]))
## Filter rows with OTHER entity type

KeyError: 'entity_type_1'

In [ ]:
features["label"] = features.apply(lambda x : f"{x['entity_type_1']}-{x['entity_type_2']}", axis=1)

In [ ]:
print(f"Number of unique entity-type-paris : {len(features['label'].unique())}")
print(f"Number of unique entity-type-paris : {features['label'].unique()}")

In [ ]:
features.drop(["entity_type_1","entity_type_2"], axis=1, inplace=True)

In [ ]:
features

## Multiclass Classification

In [86]:
features = features.groupby('label').filter(lambda x : len(x)>2)

In [87]:
features["label"].unique()

array(['ORG-LOC', 'LOC-LOC', 'PER-ORG', 'ORG-ORG', 'PER-LOC', 'PER-PER'],
      dtype=object)

In [88]:
# features_with_labels = pd.read_csv("features_"+dataset+".csv")
# features = features.groupby('label').apply(lambda x: x.sample(frac=0.2))

X = features.iloc[:,:-1]
X = X.fillna("")
y_original =  features.iloc[:,-1]

x_train_original, x_test_original, y_train_original, y_test_original = train_test_split(X, y_original, test_size=0.2, stratify=y_original)
onehot_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
x_train = onehot_encoder.fit_transform(x_train_original)
x_test = onehot_encoder.transform(x_test_original)

le = LabelEncoder()
y_train = le.fit_transform(y_train_original)
y_test = le.transform(y_test_original)
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
le_name_mapping

{'LOC-LOC': 0,
 'ORG-LOC': 1,
 'ORG-ORG': 2,
 'PER-LOC': 3,
 'PER-ORG': 4,
 'PER-PER': 5}

In [89]:
unique, count = np.unique(y_original, return_counts = True)
dict(zip(unique, count))

{'LOC-LOC': 51,
 'ORG-LOC': 135,
 'ORG-ORG': 92,
 'PER-LOC': 29,
 'PER-ORG': 30,
 'PER-PER': 30}

In [90]:
pred = logistic_regression_multiple_classes(x_train, x_test, y_train, y_test, "et")
x_test_log_pred = x_test_original
x_test_log_pred["label"] = y_test
x_test_log_pred["prediction"] = pred
x_test_log_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_logisitic_regression_prediction.csv")

--------Logistic Regression-----------


In [91]:
pred = random_forest_multiple(x_train, x_test, y_train, y_test,"et")
x_test_rf_pred = x_test_original
x_test_rf_pred["label"] = y_test
x_test_rf_pred["prediction"] = pred
x_test_rf_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_random_forest_prediction.csv")


--------Random Forest-----------


In [92]:
pred = svm_multiple("linear", x_train, x_test, y_train, y_test, "et")
x_test_svm_pred = x_test_original
x_test_svm_pred["label"] = y_test
x_test_svm_pred["prediction"] = pred
x_test_svm_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_svm_prediction.csv")

--------Support Vector Classifier linear-----------


In [93]:
pred = svm_multiple("rbf", x_train, x_test, y_train, y_test, "et")
x_test_svmrbf_pred = x_test_original
x_test_svmrbf_pred["label"] = y_test
x_test_svmrbf_pred["prediction"] = pred
x_test_svmrbf_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_svm_rbf_prediction.csv")

--------Support Vector Classifier rbf-----------


In [94]:
pred = svm_multiple("poly", x_train, x_test, y_train, y_test, "et")
x_test_svmpoly_pred = x_test_original
x_test_svmpoly_pred["label"] = y_test
x_test_svmpoly_pred["prediction"] = pred
x_test_svmpoly_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_svm_poly_prediction.csv")


--------Support Vector Classifier poly-----------


In [95]:
pred = xgboost_multiple_class(x_train, x_test, y_train, y_test, "et")
x_test_xg_pred = x_test_original
x_test_xg_pred["label"] = y_test
x_test_xg_pred["prediction"] = pred
x_test_xg_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_xgboost_prediction.csv")

In [96]:
pred = adaboost_multiple(x_train, x_test, y_train, y_test, "et")
x_test_ada_pred = x_test_original
x_test_ada_pred["label"] = y_test
x_test_ada_pred["prediction"] = pred
x_test_ada_pred.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Predictions/binary_adaboost_prediction.csv")
